In [1]:
import random
import string

def generate_order(min_items=1, max_items=10):
  """
  Generates a single order with a random number of items and unique item details.
  """
  num_items = random.randint(min_items, max_items)
  order_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
  item_list = []
  for _ in range(num_items):
    description = f"Item {len(item_list) + 1}"
    # Generate unique dimensions for each item
    dimensions = (random.uniform(5, 20), random.uniform(5, 20), random.uniform(1, 10))
    while any(dim in item_list for dim in dimensions):  # Check for duplicates
      dimensions = (random.uniform(5, 20), random.uniform(5, 20), random.uniform(1, 10))
    item_list.append({"Description": description, "Dimensions (x, y, z)": dimensions})
  return order_id, num_items, item_list

def generate_data(num_orders=100000):
  """
  Generates a list of 100,000 orders with unique item dimensions.
  """
  data = []
  num_items_list = []
  # No need to track dimensions as they are guaranteed unique

  for _ in range(num_orders):
    order_id, num_items, item_list = generate_order()
    data.append((order_id, num_items, item_list))
    num_items_list.append(num_items)

  return data, {
      "Number of Items": {
          "Min": min(num_items_list),
          "Max": max(num_items_list),
          "Average": sum(num_items_list) / len(num_items_list)
      }
  }

if __name__ == "__main__":
  data, statistics = generate_data()
  print("Generated", len(data), "orders.")
  print("Distribution Statistics:")
  for key, value in statistics.items():
    print(f"\t{key}: {value}")

  # Access data and statistics
  # data[0] - contains the first order details (order_id, num_items, item_list)


Generated 100000 orders.
Distribution Statistics:
	Number of Items: {'Min': 1, 'Max': 10, 'Average': 5.50567}


In [2]:
import json

# Assuming data is the generated list of orders
with open("orders.json", "w") as f:
  json.dump(data, f)

print("Data saved to orders.json")


Data saved to orders.json


In [3]:
import random
import string

def generate_order(min_items=1, max_items=10):
  """
  Generates a single order with a random number of items and unique item details.
  """
  num_items = random.randint(min_items, max_items)
  order_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
  item_list = []
  for _ in range(num_items):
    description = f"Item {len(item_list) + 1}"
    # Generate unique dimensions for each item
    dimensions = (random.uniform(5, 20), random.uniform(5, 20), random.uniform(1, 10))
    while any(dim in item_list for dim in dimensions):  # Check for duplicates
      dimensions = (random.uniform(5, 20), random.uniform(5, 20), random.uniform(1, 10))
    item_list.append({"Description": description, "Dimensions (x, y, z)": dimensions})
  return order_id, num_items, item_list

def generate_data(num_orders=100000):
  """
  Generates a list of 100,000 orders with unique item dimensions, and calculates distribution statistics.
  """
  data = []
  num_items_list = []
  dimensions_x = []
  dimensions_y = []
  dimensions_z = []
  
  for _ in range(num_orders):
    order_id, num_items, item_list = generate_order()
    data.append((order_id, num_items, item_list))
    num_items_list.append(num_items)
    dimensions_x.append(item_list[0]["Dimensions (x, y, z)"][0])  # Assuming first item for dimensions (can be modified)
    dimensions_y.append(item_list[0]["Dimensions (x, y, z)"][1])
    dimensions_z.append(item_list[0]["Dimensions (x, y, z)"][2])

  return data, {
      "Number of Items": {
          "Min": min(num_items_list),
          "Max": max(num_items_list),
          "Average": sum(num_items_list) / len(num_items_list)
      },
      "Item Dimensions": {
          "x": {
              "Min": min(dimensions_x),
              "Max": max(dimensions_x),
              "Average": sum(dimensions_x) / len(dimensions_x)
          },
          "y": {
              "Min": min(dimensions_y),
              "Max": max(dimensions_y),
              "Average": sum(dimensions_y) / len(dimensions_y)
          },
          "z": {
              "Min": min(dimensions_z),
              "Max": max(dimensions_z),
              "Average": sum(dimensions_z) / len(dimensions_z)
          }
      }
  }

if __name__ == "__main__":
  data, statistics = generate_data()
  print("Generated", len(data), "orders.")
  print("Distribution Statistics:")
  for key, value in statistics.items():
    print(f"\t{key}: {value}")

  # Access data and statistics
  # data[0] - contains the first order details (order_id, num_items, item_list)
  # statistics["Number of Items"]["Min"] - minimum number of items per order


Generated 100000 orders.
Distribution Statistics:
	Number of Items: {'Min': 1, 'Max': 10, 'Average': 5.49866}
	Item Dimensions: {'x': {'Min': 5.000000018940461, 'Max': 19.999507787153938, 'Average': 12.499680333938295}, 'y': {'Min': 5.000041970469566, 'Max': 19.999999708317844, 'Average': 12.503359722039859}, 'z': {'Min': 1.0000061112493257, 'Max': 9.999955149294676, 'Average': 5.505075709252739}}


In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from spektral.layers import GCNConv

def item_cnn(input_shape):
    inputs = keras.Input(shape=input_shape)  # Input for item dimensions
    if len(input_shape) == 3:  # Check if 3D input (no channels)
        x = tf.expand_dims(inputs, axis=-1)  # Add channel dimension
    else:
        x = inputs

    kernel_size = min(2, input_shape[2])  # Adjust kernel size based on input shape
    x = layers.Conv3D(filters=32, kernel_size=kernel_size, activation="relu")(x)
    x = layers.Conv3D(filters=32, kernel_size=kernel_size, activation="relu")(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)  # Downsample features
    x = layers.Conv3D(filters=64, kernel_size=kernel_size, activation="relu")(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)  # Further downsample
    x = layers.Flatten()(x)  # Convert to 1D vector for efficient combination

    return keras.Model(inputs=inputs, outputs=x) # Convert to 1D vector for efficient combination

def order_gnn(item_embedding_dim):
  """
  Defines a GNN model for learning order representations.
  """
  inputs = keras.Input(shape=(None,))  # Variable number of items in an order
  item_features = keras.Input(shape=(item_embedding_dim,))  # From item CNN

  # Define the message passing function for the GNN layer
  def message(inputs):
    node_features, edge_index = inputs
    return tf.concat([node_features, edge_index[:, 1]], axis=-1)  # Combine node features with target node index

  # Create the GNN layer with message passing function
  x = GCNConv(32, activation="relu")(inputs=[item_features, inputs])
  # Additional GNN layers can be added for more complex relationships
  return keras.Model(inputs=[inputs, item_features], outputs=x)

def combine_outputs(item_cnn_output, order_gnn_output):
  """
  Combines the outputs from the item CNN and order GNN.
  """
  x = layers.concatenate([item_cnn_output, order_gnn_output])
  return x

def predict_box(combined_features):
  """
  Predicts the optimal box dimensions and cost for a given order.
  """
  x = layers.Dense(64, activation="relu")(combined_features)  # Hidden layer for further processing
  box_dims = layers.Dense(3, activation="sigmoid")(x)  # Predict normalized box dimensions (x, y, z)
  cost_factor = layers.Dense(1)(x)  # Predict cost factor based on predicted box volume (optional)
  return box_dims, cost_factor  # Optionally return cost factor

def create_model(item_shape, item_embedding_dim):
  """
  Creates and returns the complete model with 3D CNN, GNN, and prediction layers.
  """
  item_shape_with_channels = (*item_shape, 1)  # Add channel dimension to item shape
  item_cnn_model = item_cnn(item_shape_with_channels)
  order_gnn_model = order_gnn(item_embedding_dim)

  item_inputs = keras.Input(shape=item_shape_with_channels)  # Input for item dimensions
  order_inputs = keras.Input(shape=(None,))  # Input for order (number of items)

  # Get item embeddings from the CNN
  item_embeddings = item_cnn_model(item_inputs)

  # Encode order structure as a graph (left as an exercise for customization)
  # You'll need to define how items in an order are connected as a graph
  order_graph = ...  # Define the order graph based on item relationships

  # Pass item embeddings and order graph through the GNN
  order_representation = order_gnn_model([order_graph, item_embeddings])

  # Combine outputs from CNN and GNN
  combined_features = combine_outputs(item_embeddings, order_representation)

  # Predict box dimensions and optionally cost factor
  box_dims, cost_factor = predict_box(combined_features)

  model = keras.Model(inputs=[item_inputs, order_inputs], outputs=[box_dims, cost_factor])
  return model
print("ok")

ok


In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from spektral.layers import GCNConv
def train_model(model, train_data, validation_data, epochs, batch_size, checkpoint_filepath):
  """
  Trains the model on the provided data, utilizing the entire GPU's processing power and saving checkpoints.
  """
  # Define loss function (customizable for box packing problem)
  def box_packing_loss(y_true, y_pred):
    box_dims_pred, cost_factor_pred = y_pred
    box_dims_true = y_true[0]  # Assuming true box dimensions are the first element in y_true
    # ... (Wasted space and cost calculation logic)
    return loss

  # Configure GPU usage (modify as needed based on your GPU and environment)
  physical_devices = tf.config.list_physical_devices('GPU')
  try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
  except:
    # Ignore errors if setting memory growth is not supported
    pass

  # Create a MirroredStrategy for multi-GPU training (if applicable)
  if len(physical_devices) > 1:
    strategy = tf.distribute.MirroredStrategy(devices=physical_devices)
  else:
    strategy = tf.distribute.get_strategy()  # Use default strategy (likely single GPU)

  # Wrap the model in the distribution strategy
  with strategy.scope():
    # Compile the model with loss function and optimizer
    model.compile(loss=box_packing_loss, optimizer="adam")

    # Checkpoint callback configuration
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,  # Save only model weights for efficiency
        monitor='val_loss',  # Monitor validation loss for checkpointing
        mode='min',  # Save checkpoint when validation loss improves
        verbose=1  # Print information about the checkpoint saving process
    )

    # Train the model with early stopping and checkpointing
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
    model.fit(train_data, validation_data=validation_data, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, checkpoint_callback])

# Example usage
item_shape = (10, 10, 5)  # Replace with your item dimension shape
item_embedding_dim = 64
epochs = 20  # Specify the desired number of epochs here
batch_size = 32
checkpoint_filepath = "/kaggle/working/" # Adjust based on your model architecture

from sklearn.model_selection import train_test_split

def split_data(data, test_size=0.2, val_size=0.1):
  """
  Splits the provided data into training, validation, and test sets.

  Args:
      data: A list or NumPy array containing your entire dataset.
      test_size: The proportion of data to be used for the test set (default: 0.2).
      val_size: The proportion of data to be used for the validation set (default: 0.1).

  Returns:
      A tuple containing three elements: training_data, validation_data, test_data.
  """

  # Combine training and validation sets first (optional)
  train_val_data, test_data = train_test_split(data, test_size=test_size, random_state=42)

  # Further split the training/validation set into training and validation sets (optional)
  if val_size > 0:
      train_data, validation_data = train_test_split(train_val_data, test_size=val_size/(1-test_size), random_state=42)

  return train_data, validation_data, test_data # Implement data splitting function

model = create_model(item_shape, item_embedding_dim)
train_model(model, train_data, validation_data, epochs=20, batch_size=32)

# Evaluate the model on the test data (optional)
model.evaluate(test_data)


ValueError: Exception encountered when calling layer "conv3d_8" (type Conv3D).

Negative dimension size caused by subtracting 2 from 1 for '{{node conv3d_8/Conv3D}} = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1, 1]](Placeholder, conv3d_8/Conv3D/ReadVariableOp)' with input shapes: [?,4,4,1,32], [2,2,2,32,64].

Call arguments received by layer "conv3d_8" (type Conv3D):
  • inputs=tf.Tensor(shape=(None, 4, 4, 1, 32), dtype=float32)